In [1]:
import numpy as np
import tensorflow as tf
#from tensorflow.keras.layers import TextVectorization
import pickle

2025-06-05 19:06:42.036324: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-05 19:06:42.068367: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749130602.101580   12188 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749130602.112932   12188 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749130602.144517   12188 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
def load_from_disk(path):

    with open(path, "rb") as f:
        loaded_list = pickle.load(f)

    return loaded_list

In [3]:
vocab=load_from_disk('./Vocabulary/new_vocab.pkl')

In [4]:
print(len(vocab))
print(vocab[0:10])

7826
['<SOS>', '<EOS>', '08', '1', '10', '104', '11', '12', '13', '157']


In [5]:
# vectorize_layer = TextVectorization(
#     standardize=None,
#     split=None,
#     output_mode='int',
#     vocabulary=vocab)

vectorize_layer=tf.keras.layers.StringLookup(mask_token='',vocabulary=vocab)

I0000 00:00:1749130617.154223   12188 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1767 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [6]:
vector_vocab=vectorize_layer.get_vocabulary()
print(len(vector_vocab))
print(vector_vocab[0:10])

7828
['', '[UNK]', '<SOS>', '<EOS>', '08', '1', '10', '104', '11', '12']


In [7]:
dev_captions=load_from_disk('./Data/train_captions.pkl')

In [8]:
print(len(dev_captions))
print(dev_captions[0:10])

28320
['A child in a pink dress is climbing up a set of stairs in an entry way .', 'A girl going into a wooden building .', 'A little girl climbing into a wooden playhouse .', 'A little girl climbing the stairs to her playhouse .', 'A little girl in a pink dress going into a wooden cabin .', 'A black dog and a spotted dog are fighting', 'A black dog and a tri-colored dog playing with each other on the road .', 'A black dog and a white dog with brown spots are staring at each other in the street .', 'Two dogs of different breeds looking at each other on the road .', 'Two dogs on pavement moving toward each other .']


In [9]:
import string

def preprocess_text(text):
    # Step 1: Convert all characters to lowercase
    text = text.lower()
    
    # Step 2: Remove punctuation
    # str.maketrans('', '', string.punctuation) creates a translation table
    # str.translate(...) removes all punctuation characters in that table
    text = text.translate(str.maketrans('', '', string.punctuation))

    text="<SOS> "+text+" <EOS>"
    
    return text

In [10]:
processed_dev_captions=[]
for sentence in dev_captions:
    sen=preprocess_text(sentence)
    processed_dev_captions.append(sen)

In [11]:
print(len(processed_dev_captions))
print(processed_dev_captions[0:10])

28320
['<SOS> a child in a pink dress is climbing up a set of stairs in an entry way  <EOS>', '<SOS> a girl going into a wooden building  <EOS>', '<SOS> a little girl climbing into a wooden playhouse  <EOS>', '<SOS> a little girl climbing the stairs to her playhouse  <EOS>', '<SOS> a little girl in a pink dress going into a wooden cabin  <EOS>', '<SOS> a black dog and a spotted dog are fighting <EOS>', '<SOS> a black dog and a tricolored dog playing with each other on the road  <EOS>', '<SOS> a black dog and a white dog with brown spots are staring at each other in the street  <EOS>', '<SOS> two dogs of different breeds looking at each other on the road  <EOS>', '<SOS> two dogs on pavement moving toward each other  <EOS>']


In [12]:
split_processed_dev_captions=[]
for x in processed_dev_captions:
    split=x.split()
    split_processed_dev_captions.append(split)
print('completed')

completed


In [13]:
len(split_processed_dev_captions)

28320

In [14]:
split_processed_dev_captions[0]

['<SOS>',
 'a',
 'child',
 'in',
 'a',
 'pink',
 'dress',
 'is',
 'climbing',
 'up',
 'a',
 'set',
 'of',
 'stairs',
 'in',
 'an',
 'entry',
 'way',
 '<EOS>']

In [15]:
dev_input_captions=[]
dev_label_captions=[]
for x in split_processed_dev_captions:
    dev_input_captions.append(x[0:-1])
    dev_label_captions.append(x[1:])

In [16]:
dev_input_captions[0:2]

[['<SOS>',
  'a',
  'child',
  'in',
  'a',
  'pink',
  'dress',
  'is',
  'climbing',
  'up',
  'a',
  'set',
  'of',
  'stairs',
  'in',
  'an',
  'entry',
  'way'],
 ['<SOS>', 'a', 'girl', 'going', 'into', 'a', 'wooden', 'building']]

In [17]:
processed_dev_input_captions=vectorize_layer(tf.ragged.constant(dev_input_captions, dtype=tf.string))
processed_dev_label_captions=vectorize_layer(tf.ragged.constant(dev_label_captions,dtype=tf.string))

In [18]:
print(type(processed_dev_input_captions))

<class 'tensorflow.python.ops.ragged.ragged_tensor.RaggedTensor'>


In [19]:
processed_dev_input_captions=processed_dev_input_captions.to_tensor()

In [20]:
processed_dev_input_captions.shape

TensorShape([28320, 37])

In [21]:
processed_dev_label_captions=processed_dev_label_captions.to_tensor()

In [22]:
processed_dev_label_captions.shape

TensorShape([28320, 37])

In [23]:
processed_dev_input_captions[0]

<tf.Tensor: shape=(37,), dtype=int64, numpy=
array([   2,   51, 1301, 3457,   51, 4957, 2121, 3541, 1398, 7352,   51,
       5892, 4519, 6504, 3457,  218, 2332, 7565,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0])>

In [24]:
processed_dev_label_captions[0]

<tf.Tensor: shape=(37,), dtype=int64, numpy=
array([  51, 1301, 3457,   51, 4957, 2121, 3541, 1398, 7352,   51, 5892,
       4519, 6504, 3457,  218, 2332, 7565,    3,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0])>

In [25]:
processed_dev_input_captions_np=processed_dev_input_captions.numpy()

In [26]:
type(processed_dev_input_captions_np)

numpy.ndarray

In [27]:
processed_dev_input_captions_np.shape

(28320, 37)

In [28]:
processed_dev_label_captions_np=processed_dev_label_captions.numpy()

In [29]:
type(processed_dev_label_captions_np)

numpy.ndarray

In [30]:
processed_dev_label_captions_np.shape

(28320, 37)

In [31]:
np.save('./Data/processed_train_input_captions.npy',processed_dev_input_captions_np)

In [32]:
np.save('./Data/processed_train_label_captions.npy',processed_dev_label_captions_np)